# Parsing, reading, concatenating

In [1]:
#general
import pandas as pd
import numpy as np
import os
import warnings
from pandas import ExcelWriter

#html
import requests
import json
from lxml import html
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup

#NLP
import nltk
import re
import pymorphy2
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
from nltk.util import ngrams
from nltk.corpus import stopwords 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from dateutil.parser import parse

#feature extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer

#models
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
warnings.filterwarnings('ignore')

In [2]:
#function finalnaya
def my_parser(file):
    #open file
    with open(file, encoding="utf8") as json_data:
        d = json.load(json_data)
    #берем только значения
    d1 = list(d.values()) 
    #берем только тикеты
    d2 = d1[0] 
    df = json_normalize(d2, record_path=['messages'], meta=['id'])
    df.columns = ['message', 'client_id', 'date', 'email', 'title', 'from', 'ticket_id']
    #delete useless
    df =  df.drop(df[df.message.str.contains('userAgent')].index)
    df =  df.drop(df[df.message.str.contains('DOCTYPE')].index)
    df =  df.drop(df[df.message.str.contains('HTML')].index)
    df =  df.drop(df[df.message.str.contains('BODY')].index)
    #delete nulls
    del_list = df[df.message==''].index
    df.drop(del_list, inplace = True)
    df.index = range(len(df.index))
    del_list = df[df.message=='\n\n'].index
    df.drop(del_list, inplace = True)
    df.index = range(len(df.index))
    #delete vsyakaya hren'
    df.message = df.message.map(lambda x: BeautifulSoup(BeautifulSoup(x, "lxml").get_text(), "lxml").get_text())
    return df

In [3]:
def conc(direc):
    X = []
    result=pd.DataFrame()
    for filename in os.listdir(direc):
        X.append(direc+filename)    
    for name in X:
        result = pd.concat([result, my_parser(name)], ignore_index = True) 
   # result.index = range(len(result.index))
    return result

In [4]:
df = conc('MoneyManData/')

In [5]:
df1 = df.copy()

# Очистка всего, оставляем автоматическое и удаляем тикеты имейлов

In [6]:
#удаляем в котором содержаться теги имейлов
def only_chat(df):
    drops = pd.concat([df.ticket_id[df.message.str.contains('support@moneyman.ru')],
                        df.ticket_id[df.message.str.contains('Перенаправленное сообщение')],
                       df.ticket_id[df.message.str.contains('Пересылаемое сообщение')],
                        df.ticket_id[df.message.str.contains('Наши награды')]], ignore_index = True).unique()
    df = df[~df['ticket_id'].isin(drops)]
    return (df)

def only_avto(df):
    #оставляем автоматические только
    avto = df.ticket_id[df.message.str.contains('сформировано автоматически')].unique()
    df = df[df['ticket_id'].isin(avto)]
    df.index = range(len(df.index))
    return (df)

def only_not_avto(df):
    #оставляем неавтоматические только
    avto = df.ticket_id[df.message.str.contains('сформировано автоматически')].unique()
    df = df[~df['ticket_id'].isin(avto)]
    df.index = range(len(df.index))
    return (df)

In [7]:
df = only_avto(only_chat(df))

# Создание таблицы quest-ans

In [8]:
def new_df(df):
    df1 = df.copy()
    df1.columns = ['ques', 'ans', 'dt_ques', 'dt_ans', 'ticket_id', 'email', 'client_id']
    k = 0
    for i in range(len(df)):
        if df['from'][i]==1 and df['from'][i+1]==0 and df.ticket_id[i]==df.ticket_id[i+1]:
            df1['ques'][k] = df['message'][i]
            df1['ans'][k] = df['message'][i+1]
            df1['dt_ques'][k] = df['date'][i]
            df1['dt_ans'][k] = df['date'][i+1]
            df1['ticket_id'][k] = df['ticket_id'][i]
            df1['email'][k] = df['email'][i]
            df1['client_id'][k] = df['client_id'][i]
            k+=1
    df1.index = range(len(df1.index))
    return df1[:k]

In [9]:
data = new_df(df)

In [10]:
#w = pd.Series(data.ans.value_counts())
#writer = pd.ExcelWriter('valca.xlsx')
#w.to_excel(writer,'output', index=False)
#writer.save()

# Make labels unique

In [15]:
def replasic(data):
    #если содержит это, то заменить лейбл на такой-то
    #отказ в займе
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nК сожалению, в выдаче займа Вам было отказано.\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.С уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВы можете погашать займ частями.\n\nВ случае частичного погашения проценты будут продолжать\nнакапливаться только на размер непогашенной суммы займа. Все возможные методы\nпогашения представлены на нашем сайте: \n\thttp://moneyman.ru/return-credit . Сумму для досрочного погашения на\nтекущий момент можно посмотреть в личном кабинете.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте.\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте.\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать текущий статус заявки в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nК сожалению, в выдаче займа Вам было отказано.\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.С уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #отслеживание статуса
    data.ans.replace('Здравствуйте.\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать текущий статус заявки в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать текущий статус заявки в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n')
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте.\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n')
    #Про фото для сайта
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nДля предоставления данных для входа в личный кабинет на нашем сайте пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\n\n\nВы также можете воспользоваться формой восстановления пароля: https://moneyman.ru/secure/password-recovery\n\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nДля предоставления данных для входа в личный кабинет на нашем сайте пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\nВы также можете воспользоваться формой восстановления пароля: https://moneyman.ru/secure/password-recovery\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nДля предоставления данных для входа в личный кабинет на нашем сайте пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\nВы также можете воспользоваться формой восстановления пароля: https://moneyman.ru/secure/password-recovery\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nДля предоставления данных для входа в личный кабинет на нашем сайте пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\nВы также можете воспользоваться формой восстановления пароля: https://moneyman.ru/secure/password-recovery\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    #Изменениея
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nДля изменения телефона пришлите, пожалуйста, следующую информацию:\n\n- Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец);\n\n\n\n- старый номер;\n\n- новый номер.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПришлите, пожалуйста,Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт, а также укажите, какие именно изменения нужно внести.\n\n\n\nПосле чего мы внесем необходимые изменения в Вашем личном кабинете.\n\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nДля изменения телефона пришлите, пожалуйста, следующую информацию:\n- Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец);\n\n- старый номер;\n- новый номер.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПришлите, пожалуйста,Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт, а также укажите, какие именно изменения нужно внести.\n\n\n\nПосле чего мы внесем необходимые изменения в Вашем личном кабинете.\n\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nДля изменения телефона пришлите, пожалуйста, следующую информацию:\n- Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец);\n\n- старый номер;\n- новый номер.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПришлите, пожалуйста,Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт, а также укажите, какие именно изменения нужно внести.\n\n\n\nПосле чего мы внесем необходимые изменения в Вашем личном кабинете.\n\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nПришлите, пожалуйста,Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт, а также укажите, какие именно изменения нужно внести.\n\nПосле чего мы внесем необходимые изменения в Вашем личном кабинете.\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПришлите, пожалуйста,Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт, а также укажите, какие именно изменения нужно внести.\n\n\n\nПосле чего мы внесем необходимые изменения в Вашем личном кабинете.\n\nЕсли остались какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #способы погашения займа
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nСо способами погашения займа Вы можете ознакомиться по ссылке: http://moneyman.ru/return-credit\nПожалуйста, не забудьте при оплате указать номер займа в назначении платежа. Номер займа Вы можете узнать в Личном кабинете https://moneyman.ru/secure/login\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nСо способами погашения займа Вы можете ознакомиться по ссылке: http://moneyman.ru/return-credit\n\nПожалуйста, не забудьте при оплате указать номер займа в назначении платежа. Номер займа Вы можете узнать в Личном кабинете https://moneyman.ru/secure/login\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #мы не получили платеж
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nБольшое спасибо за предоставленную информацию.\nК сожалению, мы не получили Ваш платеж. Как правило, это происходит, когда не указано назначение платежа. К примеру, вы могли указать номер вашего первого займа.\nЧтобы найти Ваш платеж, нам необходимо подтверждение оплаты. Этим может служить скан или фотография следующего:\n• Чек из терминала оплаты (Киви, Элекснет и т.д.);\n• Квитанции совершенной оплаты;\n• Выписка с банковского счета (в отсканированном виде или в формате PDF) или скриншот выписки из интернет-банка, где четко обозначено соответствующее снятие средств.\nК сожалению, мы не можем принять скриншоты мобильных приложений или SMS-сообщений в качестве подтверждения оплаты.\nМы приносим искренние извинения за неудобства, которые могла доставить эта ситуация.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nБольшое спасибо за предоставленную информацию.\n\nК сожалению, мы не получили Ваш платеж. Как правило, это происходит, когда не указано назначение платежа. К примеру, вы могли указать номер вашего первого займа.\n\nЧтобы найти Ваш платеж, нам необходимо подтверждение оплаты. Этим может служить скан или фотография следующего:\n\n• Чек из терминала оплаты (Киви, Элекснет и т.д.);\n\n• Квитанции совершенной оплаты;\n\n• Выписка с банковского счета (в отсканированном виде или в формате PDF) или скриншот выписки из интернет-банка, где четко обозначено соответствующее снятие средств.\n\nК сожалению, мы не можем принять скриншоты мобильных приложений или SMS-сообщений в качестве подтверждения оплаты.\n\nМы приносим искренние извинения за неудобства, которые могла доставить эта ситуация.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #fin problems
    data.ans.replace('Здравствуйте,\nДанное сообщение сформировано автоматически.\nМы сожалеем, что Вы столкнулись с финансовыми трудностями.\nМы готовы войти в Ваше положение и предлагаем воспользоваться услугой продления займа на срок от 1 до 4 недель.\nДля активации услуги необходимо:\n1) Авторизоваться в Личном кабинете;\n2) Нажать на "Открытый заем" и напротив строчки "Дополнительная отсрочка выплаты" нажать на "Воспользоваться".\n3) После этого необходимо выбрать срок продления (1-4 недели), подтвердить оферту кодом из СМС (статус оферты изменится на "Ожидает оплаты") и в течение трех дней произвести оплату для активации услуги.\nПожалуйста, обратите внимание, что услуга активируется только после внесенной оплаты.\nПо ссылке ниже Вы можете просмотреть, как правильно оформить продление: http://moneyman.ru/get-mikrozaim/prolongation\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте,\n\nДанное сообщение сформировано автоматически.\n\nМы сожалеем, что Вы столкнулись с финансовыми трудностями.\n\nМы готовы войти в Ваше положение и предлагаем воспользоваться услугой продления займа на срок от 1 до 4 недель.\n\nДля активации услуги необходимо:\n\n1) Авторизоваться в Личном кабинете;\n\n2) Нажать на "Открытый заем" и напротив строчки "Дополнительная отсрочка выплаты" нажать на "Воспользоваться".\n\n3) После этого необходимо выбрать срок продления (1-4 недели), подтвердить оферту кодом из СМС (статус оферты изменится на "Ожидает оплаты") и в течение трех дней произвести оплату для активации услуги.\n\nПожалуйста, обратите внимание, что услуга активируется только после внесенной оплаты.\n\nПо ссылке ниже Вы можете просмотреть, как правильно оформить продление: http://moneyman.ru/get-mikrozaim/prolongation\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nМы сожалеем, что Вы столкнулись с финансовыми трудностями.\nНаша компания может предложить Вам 2 варианта облегчения кредитной нагрузки:\n1) Вы можете воспользоваться продлением займа на срок от 1 до 4 недель. Эта услуга доступна в Вашем личном кабинете.\n2) В случае если количество дней просрочки по займу превышает 90 дней, в Вашем личном кабинете появляется возможность оформления реструктуризации.\nВы можете разбить Вашу задолженность на части. Для активации услуги необходимо:\n1) Авторизоваться в Личном кабинете;\n2) Нажать на "Открытый заем" и напротив строчки "Реструктуризация долга" нажать на "Активировать".\n3) После этого необходимо создать персональный график платежей (срок до 6 месяцев и количеством платежей в месяц (1, 2 или 4 раза)).\n4) Затем следует подтвердить оферту кодом из СМС (статус оферты изменится на "Ожидает оплаты"), и в течение трех дней произвести оплату, после чего услуга будет активирована.\nПожалуйста, обратите внимание, чтобы услуга активируется только после внесенной оплаты.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nМы сожалеем, что Вы столкнулись с финансовыми трудностями.\n\nНаша компания может предложить Вам 2 варианта облегчения кредитной нагрузки:\n\n1) Вы можете воспользоваться продлением займа на срок от 1 до 4 недель. Эта услуга доступна в Вашем личном кабинете.\n\n2) В случае если количество дней просрочки по займу превышает 90 дней, в Вашем личном кабинете появляется возможность оформления реструктуризации.\n\nВы можете разбить Вашу задолженность на части. Для активации услуги необходимо:\n\n1) Авторизоваться в Личном кабинете;\n\n2) Нажать на "Открытый заем" и напротив строчки "Реструктуризация долга" нажать на "Активировать".\n\n3) После этого необходимо создать персональный график платежей (срок до 6 месяцев и количеством платежей в месяц (1, 2 или 4 раза)).\n\n4) Затем следует подтвердить оферту кодом из СМС (статус оферты изменится на "Ожидает оплаты"), и в течение трех дней произвести оплату, после чего услуга будет активирована.\n\nПожалуйста, обратите внимание, чтобы услуга активируется только после внесенной оплаты.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #тех трудности и смс
    data.ans.replace('Здравствуйте,\n\nДанное сообщение сформировано автоматически.\n\nМы сожалеем, что Вы столкнулись с техническими трудностями при заполнении заявки.\n\nСкорее всего, Вы уже регистрировались в системе с другими контактными данными или не прошли до конца процедуру регистрации и пытаетесь повторно зарегистрироваться.\n\nНужно попробовать войти в личный кабинет на нашем сайте. Если Вы не помните логин и пароль, то пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\n\n\nМы проверим данную информацию и предоставим данные для входа в личный кабинет на нашем сайте.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте,\nДанное сообщение сформировано автоматически.\nМы сожалеем, что Вы столкнулись с техническими трудностями при заполнении заявки.\nСкорее всего, Вы уже регистрировались в системе с другими контактными данными или не прошли до конца процедуру регистрации и пытаетесь повторно зарегистрироваться.\nНужно попробовать войти в личный кабинет на нашем сайте. Если Вы не помните логин и пароль, то пришлите, пожалуйста, Вашу фотографию с Вашим паспортом в руках, где можно четко идентифицировать Вас и Ваш паспорт (см. образец):\n\nМы проверим данную информацию и предоставим данные для входа в личный кабинет на нашем сайте.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nМы сожалеем, что Вы столкнулись с техническими трудностями. Согласно нашим данным все СМС были доставлены.\nПопробуйте включить и выключить телефон. Обычно после этого недоставленные СМС приходят на телефон. Также рекомендуем проверить баланс телефона и настройки входящих сообщений (не заблокирована ли возможность приёма СМС с коротких номеров).\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nМы сожалеем, что Вы столкнулись с техническими трудностями. Согласно нашим данным все СМС были доставлены.\n\nПопробуйте включить и выключить телефон. Обычно после этого недоставленные СМС приходят на телефон. Также рекомендуем проверить баланс телефона и настройки входящих сообщений (не заблокирована ли возможность приёма СМС с коротких номеров).\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nИногда мы проводим рекламные акции с промо-кодами, о которых вы можете узнать на нашем сайте или в группе «\t Вконтакте». Промо-код можно получить, участвуя в наших акциях. С ними Вы можете ознакомиться здесь:\n\n Наш сайт: http://moneyman.ru/news \n Группе «Вконтакте»: http://vk.com/moneymanrus \nПоле «Промо-код» в анкете заявки на заем необязательно для заполнения. Если \t промо-кода у Вас нет, смело пропускайте его и продолжайте оформление заявки (не надо ничего вводить в эту графу - ни пробел, ни дефис).\n\nЕсли у Вас есть промокод, то Вам необходимо ввести его при подаче заявки из личного кабинета на шаге выбора суммы и срока (8 шаг регистрации) в соответствующее поле.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nИногда мы проводим рекламные акции с промокодами, о которых вы можете узнать на нашем сайте или в группе «Вконтакте». Промокод можно получить, участвуя в наших акциях. С ними Вы можете ознакомиться здесь:\n Наш сайт: http://moneyman.ru/news  Группе «Вконтакте»: http://vk.com/moneymanrus Поле «Промокод» в анкете заявки на заем необязательно для заполнения. Если \t промокода у Вас нет, смело пропускайте его и продолжайте оформление заявки (не надо ничего вводить в эту графу - ни пробел, ни дефис).\nЕсли у Вас есть промокод, то Вам необходимо ввести его при подаче заявки из личного кабинета на шаге выбора суммы и срока (8 шаг регистрации) в соответствующее поле.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    #акции
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nИногда мы проводим рекламные акции с промокодами, о которых вы можете узнать на нашем сайте или в группе «Вконтакте». Промокод можно получить, участвуя в наших акциях. С ними Вы можете ознакомиться здесь:\n Наш сайт: http://moneyman.ru/news  Группе «Вконтакте»: http://vk.com/moneymanrus Поле «Промокод» в анкете заявки на заем необязательно для заполнения. Если \t промокода у Вас нет, смело пропускайте его и продолжайте оформление заявки (не надо ничего вводить в эту графу - ни пробел, ни дефис).\nЕсли у Вас есть промокод, то Вам необходимо ввести его при подаче заявки из личного кабинета на шаге выбора суммы и срока (8 шаг регистрации) в соответствующее поле.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nИногда мы проводим рекламные акции с промо-кодами, о которых вы можете узнать на нашем сайте или в группе «\t Вконтакте». Промо-код можно получить, участвуя в наших акциях. С ними Вы можете ознакомиться здесь:\n\n Наш сайт: http://moneyman.ru/news \n Группе «Вконтакте»: http://vk.com/moneymanrus \nПоле «Промо-код» в анкете заявки на заем необязательно для заполнения. Если \t промо-кода у Вас нет, смело пропускайте его и продолжайте оформление заявки (не надо ничего вводить в эту графу - ни пробел, ни дефис).\n\nЕсли у Вас есть промокод, то Вам необходимо ввести его при подаче заявки из личного кабинета на шаге выбора суммы и срока (8 шаг регистрации) в соответствующее поле.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #проверьте статус
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nПожалуйста, проверьте статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае, если статус:\n- «На подтверждении»: Вы можете отказаться от займа при звонке нашего специалиста или Вы можете позвонить по номеру  +74992905582 с просьбой отменить заявку.\n- «Процессинг»: банковская карта/счет/ЯндексДеньги – деньги уже списаны, аннулировать заявку невозможно; Контакт/Юнистрим/Золотая корона - в данном случае просто не забирайте средства из Контакта/Юнистрим/Золотая Корона, и они вернутся в нашу компанию через 7 дней с закрытием текущего займа без комиссий;\n- «Ожидает погашения»: аннулировать перевод невозможно. После получения средств Вы можете произвести досрочное погашение по займу.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте.\n\nДанное сообщение сформировано автоматически.\n\nПожалуйста, проверьте статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае, если статус:\n\n- «На подтверждении»: Вы можете отказаться от займа при звонке нашего специалиста или Вы можете позвонить по номеру  +74992905582 с просьбой отменить заявку.\n\n- «Процессинг»: банковская карта/счет/ЯндексДеньги – деньги уже списаны, аннулировать заявку невозможно; Контакт/Юнистрим/Золотая корона - в данном случае просто не забирайте средства из Контакта/Юнистрим/Золотая Корона, и они вернутся в нашу компанию через 7 дней с закрытием текущего займа без комиссий;\n\n- «Ожидает погашения»: аннулировать перевод невозможно. После получения средств Вы можете произвести досрочное погашение по займу.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #одобрение заявки
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nПожалуйста, убедитесь, что Ваша заявка на займ одобрена. Статус заявки Вы можете отслеживать в личном кабинете во вкладке «Открытый займ». Личный кабинет находится по ссылке: https://moneyman.ru/secure/login.\nЕсли во вкладке Открытый займ отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nВ случае, если Вы оформили займ на банковскую карту или ЯндексКошелек, зачисление средств происходит в большинстве случаев мгновенно. При переводе на карты MasterCard могут наблюдаться проблемы со скоростью зачисления средств (до 5 банковских дней).\nЕсли Вы оформили займ, выбрав способ получения Контакт, Юнистрим или Золотая корона, Вы уже можете забирать денежные средства в ближайшем пункте выдачи по предъявлению паспорта и номера перевода, отправленного Вам в СМС-сообщении.\nПри оформлении займа на банковский счет, перечисление средств осуществляется в рабочее банковское время от 1 до 5 дней, в зависимости от скорости работы Вашего банка.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПожалуйста, убедитесь, что Ваша заявка на займ одобрена. Статус заявки Вы можете отслеживать в личном кабинете во вкладке «Открытый займ». Личный кабинет находится по ссылке: https://moneyman.ru/secure/login.\n\nЕсли во вкладке Открытый займ отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nВ случае, если Вы оформили займ на банковскую карту или ЯндексКошелек, зачисление средств происходит в большинстве случаев мгновенно. При переводе на карты MasterCard могут наблюдаться проблемы со скоростью зачисления средств (до 5 банковских дней).\n\nЕсли Вы оформили займ, выбрав способ получения Контакт, Юнистрим или Золотая корона, Вы уже можете забирать денежные средства в ближайшем пункте выдачи по предъявлению паспорта и номера перевода, отправленного Вам в СМС-сообщении.\n\nПри оформлении займа на банковский счет, перечисление средств осуществляется в рабочее банковское время от 1 до 5 дней, в зависимости от скорости работы Вашего банка.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #способы получения займа
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nСо способами получения займа Вы можете ознакомиться по ссылке: http://moneyman.ru/receive-money.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nСо способами получения займа Вы можете ознакомиться по ссылке: http://moneyman.ru/receive-money.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #отслеживать статус можете
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать текущий статус заявки в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nПожалуйста, убедитесь, что Ваша заявка находится еще на рассмотрении и по ней не принято решение (по ссылке https://moneyman.ru/secure/login , вкладка «Открытый займ» - статус "На подтверждении").\nЕсли Вы не дождались звонка нашего специалиста, то, пожалуйста, свяжитесь с ним по эл. почте card@moneyman.ru.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПожалуйста, убедитесь, что Ваша заявка находится еще на рассмотрении и по ней не принято решение (по ссылке https://moneyman.ru/secure/login , вкладка «Открытый займ» - статус "На подтверждении").\n\nЕсли Вы не дождались звонка нашего специалиста, то, пожалуйста, свяжитесь с ним по эл. почте card@moneyman.ru.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #убедитесь в фото карты
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПожалуйста, убедитесь, что Вы отправили фото запрашиваемой карты на электронную почту card@moneyman.ru\n\nИли Вы можете продублировать фото карты из Личного кабинета на нашем сайте, выбрав Тему сообщения "Фото карты".\n\nC уважением,\n\nТехподдержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nПожалуйста, убедитесь, что Вы отправили фото запрашиваемой карты на электронную почту card@moneyman.ru\nИли Вы можете продублировать фото карты из Личного кабинета на нашем сайте, выбрав Тему сообщения "Фото карты".\nC уважением,\nТехподдержка МаниМен\n', inplace = True)
    #идет процессинг
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nВаш займ находится в статусе "процессинг".\nЕсли Вы оформили заявку, выбрав способ выдачи Контакт, Юнистрим, Золотая корона, это значит, что денежные средства уже отправлены в пункт выдачи и Вы можете их получить.\nЕсли Вы оформили заявку, выбрав способ получения Банковский счет, перечисление средств на банковский счет осуществляется в рабочее банковское время от 1 до 5 дней, в зависимости от скорости работы вашего банка.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nВаш займ находится в статусе "процессинг".\n\nЕсли Вы оформили заявку, выбрав способ выдачи Контакт, Юнистрим, Золотая корона, это значит, что денежные средства уже отправлены в пункт выдачи и Вы можете их получить.\n\nЕсли Вы оформили заявку, выбрав способ получения Банковский счет, перечисление средств на банковский счет осуществляется в рабочее банковское время от 1 до 5 дней, в зависимости от скорости работы вашего банка.\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    #на рассмотрении
    data.ans.replace('Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nПожалуйста, убедитесь, что Ваша заявка еще находится на рассмотрении и по ней не принято решение (по ссылке https://moneyman.ru/secure/login , вкладка "Открытый займ" - статус "На подтверждении").\n\nЕсли Вы не дождались звонка нашего специалиста, то, пожалуйста, свяжитесь с ним по эл. почте card@moneyman.ru.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nПожалуйста, убедитесь, что Ваша заявка еще находится на рассмотрении и по ней не принято решение (по ссылке https://moneyman.ru/secure/login , вкладка "Открытый займ" - статус "На подтверждении").\nЕсли Вы не дождались звонка нашего специалиста, то, пожалуйста, свяжитесь с ним по эл. почте card@moneyman.ru.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nЕсли Ваша заявка находится "На подтверждении", то при звонке нашего специалиста попросите ее аннулировать и оформите повторно с верным способом получения.\nЕсли статус "Ожидает погашения" или "Процессинг", то изменить способ получения займа уже невозможно.\nC уважением,\nТехподдержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nЕсли Ваша заявка находится "На подтверждении", то при звонке нашего специалиста попросите ее аннулировать и оформите повторно с верным способом получения.\n\nЕсли статус "Ожидает погашения" или "Процессинг", то изменить способ получения займа уже невозможно.\n\nC уважением,\n\nТехподдержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\n\nДанное сообщение сформировано автоматически.\n\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке «Открытый займ». Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\n\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\n\nЕсли отображается статус «Ожидает погашения» или «Процессинг», значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\n\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\n\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа «На подтверждении», пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\nДанное сообщение сформировано автоматически.\nВы можете отслеживать статус Вашего займа в личном кабинете во вкладке "Открытый займ". Личный кабинет можно найти по ссылке https://moneyman.ru/secure/login.\nВ случае если статус займа "На подтверждении", пожалуйста, ожидайте звонка специалиста или свяжитесь с ним по эл. почте card@moneyman.ru.\nЕсли отображается статус "Ожидает погашения" или "Процессинг", значит, с нашей стороны денежные средства уже отправлены по указанным Вами реквизитам.\nПожалуйста, проверьте все номера телефонов, которые Вы указали при оформлении заявки, чтобы по ним можно было дозвониться и связаться с Вами или Вашим контактным лицом.\nЕсли у Вас есть какие-то вопросы, мы с удовольствием поможем.\nС уважением,\nТехническая поддержка МаниМен\n', inplace = True)
    data.ans.replace('Здравствуйте.\nДанное сообщение сформировано автоматически.\nК сожалению, в выдаче займа Вам было отказано.\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\nЕсли есть какие-то вопросы, мы с удовольствием поможем.С уважением,\nТехническая поддержка МаниМен\n','Здравствуйте!\n\nДанное сообщение сформировано автоматически.\n\nК сожалению, в выдаче займа Вам было отказано.\n\nНа окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.\n\nВы можете оценить свою кредитную историю, воспользовавшись услугой кредитный отчет: http://rating.moneyman.ru/\n\nЕсли есть какие-то вопросы, мы с удовольствием поможем.\n\nС уважением,\n\nТехническая поддержка МаниМен\n', inplace = True)
    data['freq'] = data.groupby('ans')['ans'].transform('count')
    df = data.copy()[data.freq>30]
    return df

In [16]:
df = replasic(data)

In [24]:
df[df.freq==900].ques

7        Еще раз здравствуйте! Как поменять номер карты...
16       Доброго времени суток! Не могу продолжить офор...
17       Доброго времени суток! Не могу продолжить офор...
36       \n\n\n<!-- P {margin-top:0;margin-bottom:0;} -...
39                   У меня поменялся номер что делать\n\n
40                   У меня поменялся номер что делать\n\n
41       Добрый день сменил номер телефона, как изменит...
43       Оформляю онлайн заявку,после ввода паспортных ...
61       Добрый день, менял паспорт, можите отредактиро...
65       Здравствуйте! Подскажите пытаюсь подать заявку...
69       В моей анкете перепутались данные. Вот мои дан...
71       Здравствуйте,нашел ошибку в анкете-заявлении,к...
76       ДЕНИСОВА ТАТЬЯНА АНАТОЛЬЕВНА  НОМЕР ПАСПОРТА 6...
77       Здравствуйте. Удалите этот профиль заоегистрир...
86       Потерял телефон, как сменить номер телефона в ...
87                         Как поменять паспортные данные?
114      не могу пройти оформление займа. Паспортные да.

# Feature Engineering

In [13]:
morph = pymorphy2.MorphAnalyzer()
scaler = StandardScaler()
label = LabelEncoder()

In [14]:
def tokenizer(s):
    morph = pymorphy2.MorphAnalyzer()   
    s = re.sub("[^а-яА-Я]", " ", s)    #only words ostavlyaem
    t = s.lower().split()              #маленькие буквы и разделяем
    f = []
    for j in t:
        m = morph.parse(j.replace('.',''))
        if len(m) != 0:
            wrd = m[0]
            if wrd.tag.POS not in ('NUMR','PREP','CONJ','PRCL','INTJ'):   #удаляем части речи плохие
                f.append(wrd.normal_form)
    garbagelist = ['спасибо', 'пожалуйста', 'добрый', "доброго", 'вечер', 'прошу', 'доброе', 'утро' , 'здравствуйте',
                   'здраствуйте', 'добпый', 'здрасте', "подскажите", "скажите", 'помогите','здравствовать','ваш', 'здраствовать']
    f = [word for word in f if word not in stopwords.words('russian')]   #удаляем стоп-слова
    f = [word for word in f if word not in garbagelist]   #удаляем 
    return ( " ".join(f))   

In [15]:
%%time
df.ques = df.ques.map(tokenizer)

Wall time: 10min 18s


In [16]:
#работа с датой и создание новых фич по ним
def work_with_data(df):
    df['dt_ques']=df['dt_ques'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['dt_ans']=df['dt_ans'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    df['q_day']=df['dt_ques'].apply(lambda x: x.day)
    df['q_month']=df['dt_ques'].apply(lambda x: x.month)
    df['q_weekday']=df['dt_ques'].apply(lambda x: x.weekday())
    df['a_day']=df['dt_ans'].apply(lambda x: x.day)
    df['a_month']=df['dt_ans'].apply(lambda x: x.month)
    df['a_weekday']=df['dt_ans'].apply(lambda x: x.weekday())
    #лейблим имейл
    df.email=df.email.fillna('nan')
    df['email']=label.fit_transform(df.email)
    df['client_id']=label.fit_transform(df.client_id)
    #количество обращений на клиента
    df['client_count'] = df.groupby('client_id')['client_id'].transform('count')
    #label
    df['freq']=label.fit_transform(df.freq)
    return df

In [17]:
df = work_with_data(df)
df

,ques,ans,dt_ques,dt_ans,ticket_id,email,client_id,freq,q_day,q_month,q_weekday,a_day,a_month,a_weekday,client_count
0,мочь войти личный кабинет проведение операция ...,Здравствуйте!\nДанное сообщение сформировано а...,2017-09-14 21:37:47,2017-09-14 21:37:47,5424374,4665,1231,17,14,9,3,14,9,3,1
1,мочь попасть личный кабеть помочь пожалусто,Здравствуйте!\nДанное сообщение сформировано а...,2017-09-14 23:31:15,2017-09-14 23:31:16,5424768,428,1212,17,14,9,3,14,9,3,1
2,карта получить заём,Здравствуйте!\n\nДанное сообщение сформировано...,2017-09-14 23:35:06,2017-09-14 23:35:07,5424782,2728,1232,7,14,9,3,14,9,3,1
3,отменить заявка,Здравствуйте.\n\nДанное сообщение сформировано...,2017-09-14 23:46:04,2017-09-14 23:46:05,5424816,3598,1214,10,14,9,3,14,9,3,1
4,одобрить заём банковский карта,Здравствуйте!\n\nДанное сообщение сформировано...,2017-09-15 00:21:56,2017-09-15 00:21:57,5424900,35,1233,3,15,9,4,15,9,4,1
5,прийти деньга,Здравствуйте!\n\nДанное сообщение сформировано...,2017-09-15 00:42:44,2017-09-15 00:42:45,5424960,1091,1234,9,15,9,4,15,9,4,1
6,нужно отменить заявка заём мочь подтвердить оф...,Здравствуйте.\n\nДанное сообщение сформировано...,2017-09-15 01:20:52,2017-09-15 01:20:53,5425058,2615,1236,10,15,9,4,15,9,4,2
7,поменять номер карта который приходить заёмный дс,Здравствуйте!\n\nДанное сообщение сформировано...,2017-09-15 01:34:25,2017-09-15 01:34:26,5425098,2615,1236,15,15,9,4,15,9,4,2
8,мочь изменить способ получение деньга,Здравствуйте!\n\nДанное сообщение сформировано...,2017-09-15 01:43:06,2017-09-15 01:43:06,5425130,2017,1091,0,15,9,4,15,9,4,1
9,продлить срок заём брать возвращать срок попас...,"Здравствуйте,\n\nДанное сообщение сформировано...",2017-09-15 01:58:25,2017-09-15 01:58:26,5425164,4160,1239,13,15,9,4,15,9,4,2


In [18]:
df1 = df.copy()

In [19]:
#df.drop('ans', axis = 1, inplace = True)

In [20]:
#full_quest = '\n'.join(df.ques.tolist())
#p = morph.parse('Пишет')[0].tag.POS
#print (p)

# Model

In [21]:
def get_xy(df):
    X1 = list(df.ques)
    X2 = df.drop('ques', axis = 1)
    y = list(df.freq)
    return X1, y

In [22]:
X1, y = get_xy(df)

In [23]:
#X2=scaler.fit_transform(X2)
#X2=pd.DataFrame(X2)
#X2.columns=df.columns

In [24]:
X1[0]

'мочь войти личный кабинет проведение операция писать ошибка страница найти делать хотеть продлить срок заём'

In [25]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss = 'modified_huber', penalty='elasticnet', random_state = 1))])

In [26]:
parameters = {
    #'clf__loss':
    #('log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss',
    # 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((3, 4), (4, 4),
                          (4, 5)),  #(1, 3), (2, 2), (2, 4), (3, 3),
    'vect__analyzer': ('char', 'char_wb')  # unigrams or bigrams
    #,'clf__alpha': (0.00001, 0.000001)
    #,'clf__penalty': ('l2', 'elasticnet')
}
gs = GridSearchCV(
    pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1_weighted', cv=5)

In [27]:
%%time
gs.fit(X1, y)
print("Best score: %0.3f" % gs.best_score_)
print("Best parameters set:")
best_parameters = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.3s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.0min finished


Best score: 0.888
Best parameters set:
	vect__analyzer: 'char'
	vect__max_df: 0.5
	vect__ngram_range: (4, 5)
Wall time: 1min 3s


In [28]:
def hy(x):
    if x[1]!='а':
        print ('хуйа'+x[1:])
    else:
        print ('хуй'+x[1:])

In [29]:
def generate_answer(x):
    if x.lower().find('хуесос')!=-1 or x.lower().find('пидар')!=-1 or x.lower().find('хуй')!=-1 or x.lower().find('хуйня')!=-1 or x.lower().find('пиздец')!=-1 or x.lower().find('пизда')!=-1 or x.lower().find('сука')!=-1 or x.lower().find('бля')!=-1 or x.lower().find('блять')!=-1 or x.lower().find('гады ')!=-1 or x.lower().find('пидоры')!=-1 or x.lower().find('уебки')!=-1:
        return "Здравствуйте! Сформулируйте свой вопрос без ругательств, пожалуйста."
    elif x.lower().find('как дела')!=-1 or x.lower().find('как твои дела')!=-1 or x.lower().find('как ты')!=-1:
        return "Прекрасно!"
    elif x.lower().find('сколько тебе лет')!=-1:
        return "42"
    elif x.lower().find('как тебя зовут')!=-1 or x.lower().find('кто ты')!=-1 or x.lower().find('как зовут')!=-1:
        return "Манимен-бот"
    elif len(x.split(' '))<3:
        return "Здравствуйте! Сформулируйте свой вопрос, пожалуйста."
    x = tokenizer(x)
    if (np.max(gs.predict_proba([x])) ) > 0.7:
        return (df1['ans'][df1.freq == (gs.predict([x]))[0]].any())#, np.max(gs.predict_proba([x]))
    else:
        return ("Обратитесь, пожалуйста, к оператору с Вашим вопросом :( ")

In [31]:
print (generate_answer('как оформить реструктуризацию'))

Здравствуйте!

Данное сообщение сформировано автоматически.

Мы сожалеем, что Вы столкнулись с финансовыми трудностями.

Наша компания может предложить Вам 2 варианта облегчения кредитной нагрузки:

1) Вы можете воспользоваться продлением займа на срок от 1 до 4 недель. Эта услуга доступна в Вашем личном кабинете.

2) В случае если количество дней просрочки по займу превышает 90 дней, в Вашем личном кабинете появляется возможность оформления реструктуризации.

Вы можете разбить Вашу задолженность на части. Для активации услуги необходимо:

1) Авторизоваться в Личном кабинете;

2) Нажать на "Открытый заем" и напротив строчки "Реструктуризация долга" нажать на "Активировать".

3) После этого необходимо создать персональный график платежей (срок до 6 месяцев и количеством платежей в месяц (1, 2 или 4 раза)).

4) Затем следует подтвердить оферту кодом из СМС (статус оферты изменится на "Ожидает оплаты"), и в течение трех дней произвести оплату, после чего услуга будет активирована.

Пожалу

In [114]:
#hy('')

# Save algorithm and file

In [39]:
from sklearn.externals import joblib
joblib.dump(gs.best_estimator_, 'chatbot1.pkl', compress=1);

In [38]:
writer = ExcelWriter('bots_answers.xlsx')
df1[['ans', 'freq']].drop_duplicates().to_excel(writer,'bots_answers')
writer.save() 

# Только автоматы 
## Обработка новых данных и предсказание 

In [32]:
%%time
test = replasic(new_df(only_avto(only_chat(conc('MoneyMan_nov/')))))
test['ques_old'] = test.ques
test.ques = test.ques.map(tokenizer)
X_test, y_test = get_xy(test)

Wall time: 3min 14s


In [33]:
#предсказания
test['ans_new'] = test.ques.map(lambda x: (df1['ans'][df1.freq==(gs.predict([x]))[0]].any()))
test['is_right'] = np.where(test.ans == test.ans_new, 1, 0)

In [34]:
df_wrong = test[test.is_right == 0]
df_wrong.index =range(len(df_wrong.index))

In [35]:
i=8
print ('QUESTION: \n\n', df_wrong.ques_old[i], '\n\n ANSWER_1st: \n\n', df_wrong.ans[i],'\n\n ANSWER_2nd: \n\n', df_wrong.ans_new[i])

QUESTION: 

 С личного кабинета решил продлить заим на 7 днеи сумма стоимости услуги 1512 рублеи, я оплатил с карточки 1212 рублеи и бонусами 300 рублеи, что составило 1512 рублеи но оповещение о продлении заима не поступило объясните почему? 

 ANSWER_1st: 

 Здравствуйте!

Данное сообщение сформировано автоматически.

Большое спасибо за предоставленную информацию.

К сожалению, мы не получили Ваш платеж. Как правило, это происходит, когда не указано назначение платежа. К примеру, вы могли указать номер вашего первого займа.

Чтобы найти Ваш платеж, нам необходимо подтверждение оплаты. Этим может служить скан или фотография следующего:

• Чек из терминала оплаты (Киви, Элекснет и т.д.);

• Квитанции совершенной оплаты;

• Выписка с банковского счета (в отсканированном виде или в формате PDF) или скриншот выписки из интернет-банка, где четко обозначено соответствующее снятие средств.

К сожалению, мы не можем принять скриншоты мобильных приложений или SMS-сообщений в качестве подтвержд

In [36]:
print (test.is_right.value_counts())
print ('\n\n', 'Ответы бота совпали с ответами системы в %d процентах случаев.' % (100 - 100*(test.is_right.value_counts()[0])/(test.is_right.value_counts()[1]+test.is_right.value_counts()[0])))

1    1989
0     423
Name: is_right, dtype: int64


 Ответы бота совпали с ответами системы в 82 процентах случаев.


In [44]:
writer = ExcelWriter('Not_match_with_bot.xlsx')
df_wrong[['ques_old', 'ans', 'ans_new']].drop_duplicates().to_excel(writer,'Not_match_with_bot')
writer.save() 

# Смотрим все ответы 

In [45]:
%%time
test2 = only_not_avto(only_chat(conc('MoneyMan_nov/')))
bk = test2.copy()
print (test2.ticket_id.nunique())

13601
Wall time: 47.8 s


In [46]:
%%time 
#сортировка
#test2 = test2.sort_values(by=['ticket_id', 'date'], ascending=True)
#новый дф с вопросами и ответам
test2 = new_df(test2)
#токенизер
test2['ques_old'] = test2.ques
test2.ques = test2.ques.map(tokenizer)

Wall time: 3min 34s


In [47]:
test2['ans_new'] = test2.ques.map(lambda x: (df1['ans'][df1.freq==(gs.predict([x]))[0]].any()))

In [51]:
test2.ques_old.value_counts()

\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
avto = df.ticket_id[df.message.str.contains('сформировано автоматически')].unique()
df = df[~df['ticket_id'].isin(avto)]
df.index = range(len(df.index))

In [56]:
test2

,ques,ans,dt_ques,dt_ans,ticket_id,email,client_id,ques_old,ans_new
0,смочь подать заявка заём,"Здравствуйте, Сергей Станиславович​!Следуйте, ...",2017-10-31 21:07:05,2017-11-01 05:32:56,6269428,fasting19966@gmail.com,2616510,когда я смогу подать заявку на займ?\n,Здравствуйте!\nДанное сообщение сформировано а...
1,смочь получить кредит,"Здравствуйте, Дияна Эларбековна​!Следуйте, пож...",2017-10-31 21:08:44,2017-11-01 06:39:09,6269438,nataliromanova32@gmail.com,2856538,Через сколько я смогу получить кредит,Здравствуйте!\nДанное сообщение сформировано а...
2,почему приходить ложный пароль телефон восстан...,"Здравствуйте, Андрей Михайлович!Согласно нашим...",2017-10-31 21:10:26,2017-11-01 06:25:57,6269464,andrew7910@yandex.ru,838588,Почему приходят ложные пароли на телефон для в...,Здравствуйте!\nДанное сообщение сформировано а...
3,нажатие подтвердить ничто происходить окно обн...,"Здравствуйте, Эльвира Рифкатовна.С нашей сторо...",2017-10-31 21:14:16,2017-11-01 07:09:31,6269490,Elvie27@mail.ru,1101058,"при нажатии подтвердить ничего не происходит, ...",Здравствуйте!\nДанное сообщение сформировано а...
4,вс отлично проблема страница получение деньга ...,"Здравствуйте, Михаил Владимирович​.Первый заём...",2017-10-31 21:19:58,2017-11-01 07:10:30,6269536,maikl.kudri@yandex.ru,2855708,"Всё отлично. Есть одна проблема. На странице ""...",Здравствуйте!\n\nДанное сообщение сформировано...
5,мочь офрмитя заём сказахстан находиться россия,Здравствуйте!\r\nЗаявку на заём могут оформить...,2017-10-31 21:24:27,2017-11-01 06:42:50,6269572,Mamadim65@gmail.com,2856594,Здравствуйте могу ли я офрмить займ если я ска...,Здравствуйте!\nДанное сообщение сформировано а...
6,мочь взять кредит,"Здравствуйте, Андрей Викторович.К сожалению, с...",2017-10-31 21:27:41,2017-11-01 06:49:02,6269596,Kashtan448727@bk.ru,1108560,Не могу взять кредит,Здравствуйте!\nДанное сообщение сформировано а...
7,зарег,"Здравствуйте!\r\nУточните, пожалуйста, с какой...",2017-10-31 21:27:52,2017-11-01 06:50:03,6269598,ivanova.diana96@mail.ru,2856596,а я у вас уже зарег....\n,"Здравствуйте,\nДанное сообщение сформировано а..."
8,каждый день приходить предложение заём оформит...,"Здравствуйте, Максим Сергеевич.К сожалению, со...",2017-11-01 06:08:15,2017-11-01 06:08:52,6269660,maxim.shatulo@yandex.ru,2037463,мне каждый день приходят предложения а займ оф...,Здравствуйте!\n\nДанное сообщение сформировано...
9,ночь просить сообщить информация зачисление за...,"Здравствуйте!\r\nСообщите, пожалуйста, Ваши ФИ...",2017-10-31 21:34:00,2017-11-01 06:16:52,6269662,ig1951.verizhenko@gmail.com,2856622,Доброй ночи!\nПрошу сообщить информацию о зачи...,"Здравствуйте,\n\nДанное сообщение сформировано..."


In [65]:
test2 = test2[~test2.ques_old.str.contains('Отправлено с iPhone')]
test2 = test2[~test2.ques_old.str.contains('iPhone')]
test2 = test2[~test2.ques_old.str.contains('Отправлено со')]
test2 = test2[~test2.ques_old.str.contains('Отправлено из')]
test2 = test2[~test2.ques_old.str.contains('мобильного приложения Яндекс.Почты')]
test2 = test2[~test2.ques_old.str.contains('Здравствуйте')]
test2 = test2[~test2.ques_old.str.contains('Отправлено с')]

In [48]:
i = 20
print ('QUESTION: \n\n', test2.ques_old[i], '\n\n ANSWER_SUP: \n\n', test2.ans[i],'\n\n ANSWER_BOT: \n\n', test2.ans_new[i])

QUESTION: 

 Здравствуйте,подскажите получили ли вы фотографию моей карты на почту 

 ANSWER_SUP: 

 Здравствуйте, Николай Сергеевич.К сожалению, в выдаче займа Вам было отказано.
На окончательное решение при рассмотрении заявки влияет множество факторов. В частности, проверяется информация, которую Вы указывали в анкете, номер телефона работодателя, контактный/мобильный, кредитная история, размер подтвержденного дохода, сумма/срок займа и др.
Если есть какие-то вопросы, мы с удовольствием поможем.
С уважением,
Техническая поддержка МаниМен
 

 ANSWER_BOT: 

 Здравствуйте!

Данное сообщение сформировано автоматически.

Со способами получения займа Вы можете ознакомиться по ссылке: http://moneyman.ru/receive-money.

Если есть какие-то вопросы, мы с удовольствием поможем.

С уважением,

Техническая поддержка МаниМен



In [66]:
writer = ExcelWriter('Not_answered_by_system.xlsx')
test2[['ques_old', 'ans_new']].drop_duplicates().to_excel(writer,'Not_answered_by_system')
writer.save() 

# Writing

In [41]:
#df.to_csv("output_1.csv", index=False)

In [42]:
#writer = pd.ExcelWriter('output_1.xlsx')
#df.to_excel(writer,'output', index=False)
#writer.save()

In [43]:
#print ('\n'.join(df['message'].tolist()))

# Выгрузка алгоритма

In [1]:
from sklearn.externals import joblib
joblib.dump(grid.best_estimator_, 'filename.pkl')
joblib.dump(grid_search.best_estimator_, 'filename.pkl', compress = 1)